In [1]:
from bs4 import BeautifulSoup
import pandas as pd
from datetime import datetime, timedelta
from itertools import combinations
import json
import requests
import math
from typing import Optional, List, Dict

from pydantic import BaseModel
from typing import Mapping
import numpy as np
import polars as pl

In [2]:
def formalize_data (bolillas_str: str) -> tuple[str, tuple[int]]:
  arr = bolillas_str.split(" ")
  arr.sort()
  combo = tuple(int(a) for a in arr)
  id = "".join(arr)
  return id, combo

a = formalize_data("09 12 27 33 24 30")
print(a)

('091224273033', (9, 12, 24, 27, 30, 33))


In [4]:
# Lista de números primos entre 1 y 50
PRIME_NUMBERS = {2, 3, 5, 7, 11, 13, 17, 19, 23, 29, 31, 37, 41, 43, 47}

# Configuración
TOTAL_NUMBERS = 50
CHOOSE = 6

# Combinaciones pasadas
past = [
  [1, 2, 3, 4, 5, 6],
  [1, 2, 3, 4, 5, 7],
  [1, 2, 3, 4, 5, 8]
]

# Rangos de valores de las bolillas b1 a b6
MIN_B_VALUES = [1, 2, 3, 4, 5, 6]
MAX_B_VALUES = [45, 46, 47, 48, 49, 50]

# Función para determinar si un número es primo
# Utilizando una tabla precalculada para mayor eficiencia
def is_prime(num):
  return num in PRIME_NUMBERS

# Función para determinar si una combinación es "rara"
def is_rare_combination(combo) -> tuple[bool, str]:
  # Regla 1: Todos los números son consecutivos
  if all(combo[i] + 1 == combo[i + 1] for i in range(len(combo) - 1)):
    return True, "CONSECUTIVES"

  # Regla 2: Saltos regulares (diferencia constante entre números)
  differences = [combo[i + 1] - combo[i] for i in range(len(combo) - 1)]
  if len(set(differences)) == 1:
    return True, "REGULAR_SKIPS"

  # Regla 3: Saltos consecutivos iguales
  if any(differences[i] == differences[i + 1] for i in range(len(differences) - 1)):
    return True, "REGULAR_JUMPS"

  # Regla 4: Máximo 3 números primos
  prime_count = sum(1 for num in combo if is_prime(num))
  if prime_count > 3:
    return True, "MAX_PRIMES"

  # Regla 5: Al menos un número par
  if not any(num % 2 == 0 for num in combo):
    return True, "HAVENT_EVENS"

  # Regla 6: No debe estar en combinaciones pasadas
  if list(combo) in past:
    return True, "PAST"

  return False, ""


# Calcular probabilidades de izquierda a derecha
def calculate_left_scores(combo):
  left_scores = []

  for i, value in enumerate(combo):
    if i == 0:
      possible_values = combo[i + 1] - 1
    else:
      possible_values = MAX_B_VALUES[i] - (combo[i - 1] + 1) + 1
    
    left_scores.append(1 / possible_values)

  return left_scores

# Calcular probabilidades de derecha a izquierda
def calculate_right_scores(combo):
  right_scores = []

  for i, value in enumerate(combo):
    if (i == CHOOSE - 1):
      possible_values = MAX_B_VALUES[i] - (combo[i - 1] + 1) + 1
    else:
      possible_values = (combo[i + 1] - 1) - MIN_B_VALUES[i] + 1
    
    right_scores.append(1 / possible_values)

  return right_scores




In [5]:
l = [0.0588, 0.0303, 0.0345, 0.0455, 0.0526, 0.0588]
r = [0.0588, 0.0417, 0.0370, 0.0345, 0.0333, 0.0588]

def check_shared_values (left_arr: List[float], right_arr: List[float]):
  shared_values = []
  if (left_arr[0] == right_arr[CHOOSE-1]):
    shared_values.append(left_arr[0])
  if (left_arr[CHOOSE-1] == right_arr[0]):
    shared_values.append(left_arr[CHOOSE-1])

  for i in range(len(left_arr)):
    for j in range(1, len(right_arr) - 1):
      if left_arr[i] == right_arr[j]:
        shared_values.append(left_arr[i])
  return shared_values

# iniciar comparacion desde aqui

In [6]:
# url = 'https://resultados.intralot.com.pe/i.do?m=historico&t=0&s=41'
url = 'https://resultados.latinka.com.pe/i.do?m=historico&t=0&s=41'
# response = requests.get(url, verify=False)
response = requests.get(url)
soup = BeautifulSoup(response.text, 'html.parser')

historico = soup.find(id = 'historico')
rows = historico.find_all('tr')

In [7]:
lst = []
for row in rows:
  cols = [data.text.strip() for data in row.find_all('td')]
  
  id, bolillas_int = formalize_data(bolillas_str=cols[2])
  cols.append(id)
  cols.append(bolillas_int)
  cols.extend([b for b in bolillas_int])
  
  is_rare, reason = is_rare_combination(bolillas_int)
  cols.append(is_rare)
  cols.append(reason)

  left_scores = [round(b, 4) for b in calculate_left_scores(bolillas_int)]
  right_scores = [round(b, 4) for b in calculate_right_scores(bolillas_int)]
  lr_scores = [round(left + right,4) for left, right in zip(left_scores, right_scores)]
  row_score = round(sum(lr_scores),4)

  shared_values = list(set(left_scores[0:5]) & set(right_scores[1:6]))
  shared_values = [round(s, 4) for s in shared_values]
  cols.extend([score for score in left_scores])
  cols.extend([score for score in right_scores])
  # cols.extend(lr_scores)
  cols.append(shared_values)
  cols.append(len(shared_values))
  cols.append(row_score)
  
  lst.append(cols)

with open('./tinkache.json', 'w') as file: 
  json.dump(lst, file)

dfRows = pl.DataFrame(
  lst,
  schema=[
    'fecha', 'sorteo', 'bolillas', 'yapa', 'adicionales', 'sorteo_extra',
    'id', 'combo', 'b1', 'b2', 'b3', 'b4', 'b5', 'b6',
    'is_rare_combination', 'reason',
    "score_left_b1", "score_left_b2", "score_left_b3", "score_left_b4", "score_left_b5", "score_left_b6",
    "score_r8_b1", "score_r8_b2", "score_r8_b3", "score_r8_b4", "score_r8_b5", "score_r8_b6",
    # "score_lr_b1", "score_lr_b2", "score_lr_b3", "score_lr_b4", "score_lr_b5", "score_lr_b6",
    "shared_values", "shared_values_count",
    "score_row"
  ],
  schema_overrides={
    'combo': pl.Array(pl.UInt8, 6),
    'b1': pl.UInt8,
    'b2': pl.UInt8,
    'b3': pl.UInt8,
    'b4': pl.UInt8,
    'b5': pl.UInt8,
    'b6': pl.UInt8,
    'shared_values_count': pl.UInt8
  },
  orient='row'
)

display(dfRows.head(10))


fecha,sorteo,bolillas,yapa,adicionales,sorteo_extra,id,combo,b1,b2,b3,b4,b5,b6,is_rare_combination,reason,score_left_b1,score_left_b2,score_left_b3,score_left_b4,score_left_b5,score_left_b6,score_r8_b1,score_r8_b2,score_r8_b3,score_r8_b4,score_r8_b5,score_r8_b6,shared_values,shared_values_count,score_row
str,str,str,str,str,str,str,"array[u8, 6]",u8,u8,u8,u8,u8,u8,bool,str,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,list[f64],u8,f64
"""05/02/2025""","""1166""","""19 12 27 33 24 30""","""17""","""07""","""Promoción Sí o Sí""","""121924273033""","[12, 19, … 33]",12,19,24,27,30,33,true,"""REGULAR_JUMPS""",0.0556,0.0294,0.0357,0.0417,0.0455,0.05,0.0556,0.0455,0.0417,0.0385,0.0357,0.05,"[0.0455, 0.0417, 0.0357]",3,0.5249
"""02/02/2025""","""1165""","""20 48 14 12 28 04""","""02""","""40""","""Promoción Sí o Sí""","""041214202848""","[4, 12, … 48]",4,12,14,20,28,48,false,"""""",0.0909,0.0238,0.0286,0.0294,0.0345,0.0455,0.0909,0.0833,0.0588,0.0417,0.0233,0.0455,[],0,0.5962
"""29/01/2025""","""1164""","""33 23 17 13 24 12""","""26""","""38 39 28 29""","""Promoción Sí o Sí""","""121317232433""","[12, 13, … 33]",12,13,17,23,24,33,false,"""""",0.0833,0.0294,0.0294,0.0323,0.0385,0.0385,0.0833,0.0667,0.05,0.05,0.0357,0.0385,[0.0385],1,0.5756
"""26/01/2025""","""1163""","""33 26 35 30 13 18""","""05""","""27 10""","""Promoción Sí o Sí""","""131826303335""","[13, 18, … 35]",13,18,26,30,33,35,false,"""""",0.0588,0.0303,0.0345,0.0455,0.0526,0.0588,0.0588,0.0417,0.037,0.0345,0.0333,0.0588,"[0.0345, 0.0588]",2,0.5446
"""22/01/2025""","""1162""","""21 10 49 31 32 06""","""14""","""39 29 36""","""Promoción Sí o Sí""","""061021313249""","[6, 10, … 49]",6,10,21,31,32,49,false,"""""",0.1111,0.025,0.027,0.037,0.0556,0.0556,0.1111,0.0526,0.0357,0.0357,0.0227,0.0556,[0.0556],1,0.6247
"""19/01/2025""","""1161""","""23 44 45 05 02 01""","""16""","""24 19""","""Promoción Sí o Sí""","""010205234445""","[1, 2, … 45]",1,2,5,23,44,45,false,"""""",1.0,0.0222,0.0222,0.0233,0.0385,0.1667,1.0,0.3333,0.05,0.025,0.025,0.1667,[],0,2.8729
"""15/01/2025""","""1160""","""36 16 47 38 19 46""","""32""","""18 24 43 08""","""Promoción Sí o Sí""","""161936384647""","[16, 19, … 47]",16,19,36,38,46,47,false,"""""",0.0556,0.0333,0.0357,0.0833,0.0909,0.25,0.0556,0.0294,0.0286,0.0238,0.0238,0.25,[],0,0.96
"""12/01/2025""","""1159""","""49 32 41 44 26 34""","""14""","""43""","""Promoción Sí o Sí""","""263234414449""","[26, 32, … 49]",26,32,34,41,44,49,false,"""""",0.0323,0.05,0.0667,0.0714,0.125,0.1667,0.0323,0.0312,0.0263,0.025,0.0227,0.1667,[],0,0.8163
"""08/01/2025""","""1158""","""40 25 24 47 20 44""","""31""","""46 30 09""","""Promoción Sí o Sí""","""202425404447""","[20, 24, … 47]",20,24,25,40,44,47,false,"""""",0.0435,0.0385,0.0435,0.0435,0.1111,0.1667,0.0435,0.0435,0.027,0.025,0.0238,0.1667,[0.0435],1,0.7763


In [8]:
dfRows.group_by(
  'reason'
).agg(pl.count('reason').alias('count'))

reason,count
str,u32
"""REGULAR_JUMPS""",611
"""MAX_PRIMES""",95
"""HAVENT_EVENS""",9
"""""",1733


In [9]:
dfRows.group_by(
  'shared_values_count'
).agg(pl.count('shared_values_count').alias('count'))

shared_values_count,count
u8,u32
3,31
0,1227
4,3
1,952
2,235


In [24]:
def show_b_stats (col: str):
  df_viz = dfRows.group_by(
    col
  ).agg(pl.count(col).alias('count'))

  chart = df_viz.plot.bar(
    x=col,
    y='count',
    color=col,
  )

  display(df_viz)
  display(chart)



In [35]:
show_b_stats('score_left_b1')
show_b_stats('score_r8_b1')


score_left_b1,count
f64,u32
0.125,142
0.0476,54
0.0588,93
0.0333,8
0.0769,114
…,…
0.0385,17
0.0909,135
0.0526,64


alt.Chart(...)

score_r8_b1,count
f64,u32
0.125,142
0.0476,54
0.0333,8
0.0769,114
0.0588,93
…,…
0.0385,17
0.0909,135
0.0526,64


alt.Chart(...)

In [36]:
show_b_stats('score_left_b2')
show_b_stats('score_r8_b2')

score_left_b2,count
f64,u32
0.0233,233
0.0476,7
0.025,160
0.0769,1
0.0588,1
…,…
0.0227,298
0.0385,26
0.0556,4


alt.Chart(...)

score_r8_b2,count
f64,u32
0.125,81
0.0476,94
0.0333,32
0.0769,94
0.0588,126
…,…
0.0385,67
0.0909,111
0.0526,115


alt.Chart(...)

In [37]:
show_b_stats('score_left_b3')
show_b_stats('score_r8_b3')

score_left_b3,count
f64,u32
0.0233,99
0.025,115
0.0476,31
0.0769,5
0.0333,88
…,…
0.0227,71
0.0909,2
0.0526,13


alt.Chart(...)

score_r8_b3,count
f64,u32
0.025,5
0.125,18
0.0476,116
0.0769,57
0.0333,91
…,…
0.0244,1
0.0909,52
0.0556,90


alt.Chart(...)

In [38]:
show_b_stats('score_left_b4')
show_b_stats('score_r8_b4')

score_left_b4,count
f64,u32
0.125,2
0.025,56
0.0233,17
0.0476,76
0.0769,26
…,…
0.0385,94
0.0909,3
0.0556,54


alt.Chart(...)

score_r8_b4,count
f64,u32
0.0476,78
0.0233,2
0.025,46
0.125,3
0.0333,121
…,…
0.0244,8
0.0909,13
0.0526,64


alt.Chart(...)

In [39]:
show_b_stats('score_left_b5')
show_b_stats('score_r8_b5')

score_left_b5,count
f64,u32
0.0476,121
0.125,18
0.0233,2
0.025,15
0.0588,99
…,…
0.0244,1
0.0909,52
0.0526,113


alt.Chart(...)

score_r8_b5,count
f64,u32
0.0233,16
0.0476,26
0.025,297
0.0333,102
0.0588,8
…,…
0.0385,57
0.0227,6
0.0526,8


alt.Chart(...)

In [40]:
show_b_stats('score_left_b6')
show_b_stats('score_r8_b6')

score_left_b6,count
f64,u32
0.025,1
0.125,92
0.0476,87
0.0769,141
0.0333,34
…,…
0.0385,63
0.0909,124
0.0556,124


alt.Chart(...)

score_r8_b6,count
f64,u32
0.0476,87
0.025,1
0.125,92
0.0588,103
0.0769,141
…,…
0.1667,46
0.0909,124
0.0526,108


alt.Chart(...)